# Criação das tabelas Fato da camada Gold

## Carregando as bibliotecas

In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession


StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 8, Finished, Available, Finished)

## SPARK SESSION

In [7]:
spark = SparkSession.builder.appName("TransformationsGoldLayerFacts").getOrCreate()

StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 9, Finished, Available, Finished)

## Importação dos dados da camada Silver
Aqui temos algumas opções: 
- Importar do storage do Data Lake (landing zone);
- Importar de um shortcut;
- Importar da Tabela (Table) propagada do Lakehouse no schema broze;

Nos exemplos abaixo usaremos a terceira opção por simplicidade. 

In [8]:
# Usando Variaveis para parametrizar os codigos:
INPUT_LAYER = "silver"
WORKSPACE_NAME = spark.conf.get("trident.workspace.name")
LAKEHOUSE_NAME = spark.conf.get("trident.lakehouse.name")
OUTPUT_LAYER = "gold"

# Diretorio de saida completo
INPUT_PATH = f"abfss://{WORKSPACE_NAME}@onelake.dfs.fabric.microsoft.com/{LAKEHOUSE_NAME}.Lakehouse/Tables/{INPUT_LAYER}"
OUTPUT_PATH = f"abfss://{WORKSPACE_NAME}@onelake.dfs.fabric.microsoft.com/{LAKEHOUSE_NAME}.Lakehouse/Tables/{OUTPUT_LAYER}"
# Diretorio de saida curto
# Tables/{OUTPUT_LAYER}/olist_customers_dataset.csv

StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 10, Finished, Available, Finished)

In [9]:
order_items_df = spark.read.format("delta")\
        .load(f"{INPUT_PATH}/order_items_silver")
order_payments_df = spark.read.format("delta")\
        .load(f"{INPUT_PATH}/order_payments_silver")
order_reviews_df = spark.read.format("delta")\
        .load(f"{INPUT_PATH}/order_reviews_silver")
orders_df = spark.read.format("delta")\
        .load(f"{INPUT_PATH}/orders_silver")


StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 11, Finished, Available, Finished)

## Tabelas Fatos

###  Carreganado as tabelas Orders, Order_Items, Order_Payments, Order_Reviews 

#### Objetivo:
A tabela FactSales é a tabela de fatos transacionais principal na Camada Gold. 
Ela armazena transações de vendas detalhadas no nível mais granular, vinculando clientes, produtos, pedidos e vendedores.

**Tabelas Fonte (Camada Silver):**  
- silver.orders_silver
- silver.order_items_silver
- silver.order_payments_silver
- silver.order_reviews_silver

#### Transformações Aplicadas  
- Unindo Múltiplas Tabelas → Consolidando dados de pedidos, pagamentos e avaliações
- Gerando Chaves Substitutas → fact_sales_sk como um identificador único
- Garantindo Integridade Referencial → Vinculando às tabelas Dim (customer_sk, product_sk, etc.)
- Tratando Valores Ausentes → Valores padrão para dados ausentes
- Formatando Colunas de Data → Padronizando timestamps
- Agregando Métricas Chave → Receita, desconto e valores de pagamento
- Otimizando Desempenho de Consulta → Salvando no formato Delta

#### Explicação das Transformações  
- **Junção de Múltiplas Tabelas** → Combina pedidos, itens de pedidos, pagamentos de pedidos e avaliações de pedidos.
- **Geração de Chaves Substitutas** → fact_sales_sk para identificação única de linha.
- **Garantindo Integridade Referencial** → customer_sk, product_sk, seller_sk vinculam-se às tabelas de dimensão.
- **Tratamento de Valores Ausentes** → Preenchendo NULLs com valores padrão (por exemplo, "Desconhecido" para tipo de pagamento ausente).
- **Formatação de Colunas de Data** → Padronizando formatos de data.
- **Agregação de Métricas Chave** → Valor de vendas, custo de envio, valor de desconto, sentimento da avaliação.
- **Otimização de Desempenho de Consulta** → Salvando como formato Delta para buscas mais rápidas.


In [10]:
# Juntar Tabelas
fact_sales_df = orders_df \
    .join(order_items_df, "order_id", "inner") \
    .join(order_payments_df, "order_id", "left") \
    .join(order_reviews_df, "order_id", "left") \
    .select(
        monotonically_increasing_id().cast("string").alias("fact_sales_sk"),  # Gerando Chave Surrogate
        col("order_id"),
        col("customer_id").cast("string").alias("customer_sk"),  # Vinculando a DimCustomer
        col("product_id").cast("string").alias("product_sk"),  # Vinculando a DimProduct
        col("seller_id").cast("string").alias("seller_sk"),  # Vinculando a DimSeller
        col("order_status"),
        col("order_purchase_timestamp").alias("purchase_date"),
        col("order_delivered_customer_date").alias("delivery_date"),
        col("price").alias("sales_amount"),
        col("freight_value").alias("shipping_cost"),
        col("payment_value").alias("payment_amount"),
        col("payment_type"),
        col("review_score").cast("int").alias("review_score"),
        when(col("review_score") >= 4, "Positive")
        .when(col("review_score") == 3, "Neutral")
        .otherwise("Negative").alias("review_sentiment")
    )

# Tratar Valores Ausentes
fact_sales_df = fact_sales_df.fillna({
    "review_score": 0,
    "review_sentiment": "Unknown",
    "payment_type": "Unknown",
    "delivery_date": "1900-01-01"
})

# Salvar como Tabela Delta na Camada Gold
fact_sales_df.write.format("delta")\
                    .mode("overwrite")\
                    .option("path", f"{OUTPUT_PATH}/fact_sales")\
                    .save()

StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 12, Finished, Available, Finished)

# Agregando dados da Fato Vendas

## Objetivo
A tabela FactSalesAggregation é uma tabela de fatos agregada que fornece resumos pré-computados do desempenho de vendas. Esta tabela melhora o desempenho de consultas para relatórios e análises.  

## Tabelas Fonte (Camada Gold)
- FactSales (Camada Gold)

### Transformações Aplicadas
- **Agregação de Dados para Relatórios** → SUM, AVG, COUNT em métricas chave
- **Geração de Chaves Substitutas** → fact_sales_agg_sk para identificação de linha
- **Agregação Baseada em Datas** → Cálculos de receita mensal e anual
- **Otimização para Desempenho** → Particionamento por ano_mês para consultas rápidas

## Explicação das Transformações
- **Agregação de Dados para Relatórios** → SUM(total_sales), SUM(total_shipping_cost), SUM(total_discount), AVG(review_score), COUNT(total_orders)
- **Geração de Chaves Substitutas** → fact_sales_agg_sk identifica exclusivamente cada linha.
- **Agregação Baseada em Datas** → Vendas Anuais e Mensais → A coluna year_month ajuda na análise de séries temporais.
- **Otimização para Desempenho** → Particionamento por year_month para consultas mais rápidas em dashboards.



In [11]:
# Agregação de dados de venda por data e mês
fact_sales_agg_df = fact_sales_df.groupBy(
        year(col("purchase_date")).alias("year"),
        month(col("purchase_date")).alias("month"),
        concat(year(col("purchase_date")), lit("-"), month(col("purchase_date"))).alias("year_month"),
        col("product_sk"),
        col("seller_sk")
    ) \
    .agg(
        sum(col("sales_amount")).alias("total_sales"),
        sum(col("shipping_cost")).alias("total_shipping_cost"),
        avg(col("review_score")).alias("avg_review_score"),
        count(col("fact_sales_sk")).alias("total_orders")
    ) \
    .withColumn("fact_sales_agg_sk", monotonically_increasing_id())  # Gerando a chave Surrogate

# Salvando como tabela Delta na camada Gold (Particionada para consultas mais rápidas)
fact_sales_agg_df.write.format("delta")\
                        .mode("overwrite")\
                        .partitionBy("year_month")\
                        .option("path", f"{OUTPUT_PATH}/fact_sales_agg")\
                        .save()

StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 13, Finished, Available, Finished)

# Criação da tabela BridgeOrderItems
Aqui está a tradução para PTBR:

## Objetivo
A tabela BridgeOrderItems é uma tabela de ponte que resolve a relação muitos-para-muitos entre Pedidos e Produtos. Em um conjunto de dados de e-commerce, um único pedido pode conter vários produtos, e um produto pode fazer parte de vários pedidos. Esta tabela facilita consultas eficientes das relações pedido-produto.

- Tabelas Fonte (Camada Gold):
- FactSales (Camada Gold)
- DimProduct (Camada Gold)
- DimOrderStatus (Camada Gold)

### Transformações Aplicadas
- **Resolvendo Relação Muitos-para-Muitos** → Tabela de ponte entre Pedidos e Produtos
- **Geração de Chaves Substitutas** → bridge_order_items_sk para exclusividade
- **Retenção de Atributos Críticos para Negócios** → order_sk, product_sk, order_status_sk, quantidade, preço_unitário, preço_total
- **Otimização para Desempenho de Consulta** → Armazenamento como Tabela Delta

### Explicação das Transformações
- **Resolvendo Relação Muitos-para-Muitos** → Liga Pedidos e Produtos através de uma tabela de ponte.
- **Geração de Chaves Substitutas** → bridge_order_items_sk identifica exclusivamente cada linha. 
- **Retenção de Atributos Críticos para Negócios**:  
    - **order_sk** → Liga-se a DimOrderStatus.
    - **product_sk** → Liga-se a DimProduct.
    - **quantidade, preço_unitário, preço_total** → Atributos essenciais de vendas.

- **Otimização para Desempenho** → Armazenamento como Tabela Delta para consultas rápidas e indexação.


In [13]:
from pyspark.sql.functions import col, monotonically_increasing_id

# Selecionando as colunas necessárias da tabela FactSales
bridge_order_items_df = fact_sales_df.select(
    col("fact_sales_sk"),
    col("product_sk"),
    col("order_status"),
    col("sales_amount").alias("total_price")
)

# Gerando a chave Surrogate para a tabela Bridge
bridge_order_items_df = bridge_order_items_df.withColumn("bridge_order_items_sk", monotonically_increasing_id())

# Salvando como tabela Delta na camada Gold 
bridge_order_items_df.write.format("delta")\
                        .mode("overwrite")\
                        .option("path", f"{OUTPUT_PATH}/bridge_order_items")\
                        .save()


StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 15, Finished, Available, Finished)